# Extraction and cleaning of Major Crime Indicators data in Toronto
#### Source: Toronto Police API
#### Note: Decision made to use the CSV file to have the complete dataset as the API is limited to 2000 records per extraction

In [1]:
# Dependencies
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import scipy.stats as st

In [2]:
# Store filepath into variable
mci_toronto_csv = Path("../Data Retrieval & Resources/MCI_Open_Data.csv")

In [3]:
# Read in file
mci_df = pd.read_csv(mci_toronto_csv)
mci_df.head()

,X,Y,OBJECTID,EVENT_UNIQUE_ID,REPORT_DATE,OCC_DATE,REPORT_YEAR,REPORT_MONTH,REPORT_DAY,REPORT_DOY,...,UCR_CODE,UCR_EXT,OFFENCE,MCI_CATEGORY,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,LONG_WGS84,LAT_WGS84
0,-79.425896,43.757346,1,GO-20141262074,2014/01/01 05:00:00+00,1998/06/01 04:00:00+00,2014,January,1,1,...,1480,110,Administering Noxious Thing,Assault,38,Lansing-Westgate,38,Lansing-Westgate (38),-79.425896,43.757346
1,-79.350229,43.646293,2,GO-20141260701,2014/01/01 05:00:00+00,2014/01/01 05:00:00+00,2014,January,1,1,...,2120,200,B&E,Break and Enter,70,South Riverdale,70,South Riverdale (70),-79.350229,43.646293
2,-79.376497,43.666423,3,GO-20141260889,2014/01/01 05:00:00+00,2014/01/01 05:00:00+00,2014,January,1,1,...,1430,100,Assault,Assault,74,North St.James Town,74,North St.James Town (74),-79.376497,43.666423
3,-85.488744,0.000000,4,GO-20141260973,2014/01/01 05:00:00+00,2014/01/01 05:00:00+00,2014,January,1,1,...,2130,210,Theft Over,Theft Over,NSA,NSA,NSA,NSA,-85.488744,0.000000
4,-79.344839,43.678946,5,GO-20141261050,2014/01/01 05:00:00+00,2014/01/01 05:00:00+00,2014,January,1,1,...,1430,100,Assault,Assault,69,Blake-Jones,66,Danforth (66),-79.344839,43.678946


In [4]:
# Get the column names. 
mci_df.columns

Index(['X', 'Y', 'OBJECTID', 'EVENT_UNIQUE_ID', 'REPORT_DATE', 'OCC_DATE',
       'REPORT_YEAR', 'REPORT_MONTH', 'REPORT_DAY', 'REPORT_DOY', 'REPORT_DOW',
       'REPORT_HOUR', 'OCC_YEAR', 'OCC_MONTH', 'OCC_DAY', 'OCC_DOY', 'OCC_DOW',
       'OCC_HOUR', 'DIVISION', 'LOCATION_TYPE', 'PREMISES_TYPE', 'UCR_CODE',
       'UCR_EXT', 'OFFENCE', 'MCI_CATEGORY', 'HOOD_158', 'NEIGHBOURHOOD_158',
       'HOOD_140', 'NEIGHBOURHOOD_140', 'LONG_WGS84', 'LAT_WGS84'],
      dtype='object')

In [5]:
# Keep columns we're interested in
mci_df=mci_df[["EVENT_UNIQUE_ID", "REPORT_DATE", "OCC_DATE", "OCC_YEAR", "OCC_MONTH", "OCC_DAY", "OCC_DOY", 
               "OCC_DOW", "PREMISES_TYPE", "MCI_CATEGORY", "HOOD_158", "NEIGHBOURHOOD_158", "LONG_WGS84", "LAT_WGS84"]]
mci_df.head()

,EVENT_UNIQUE_ID,REPORT_DATE,OCC_DATE,OCC_YEAR,OCC_MONTH,OCC_DAY,OCC_DOY,OCC_DOW,PREMISES_TYPE,MCI_CATEGORY,HOOD_158,NEIGHBOURHOOD_158,LONG_WGS84,LAT_WGS84
0,GO-20141262074,2014/01/01 05:00:00+00,1998/06/01 04:00:00+00,NaN,NaN,NaN,NaN,NaN,Apartment,Assault,38,Lansing-Westgate,-79.425896,43.757346
1,GO-20141260701,2014/01/01 05:00:00+00,2014/01/01 05:00:00+00,2014.0,January,1.0,1.0,Wednesday,Commercial,Break and Enter,70,South Riverdale,-79.350229,43.646293
2,GO-20141260889,2014/01/01 05:00:00+00,2014/01/01 05:00:00+00,2014.0,January,1.0,1.0,Wednesday,Apartment,Assault,74,North St.James Town,-79.376497,43.666423
3,GO-20141260973,2014/01/01 05:00:00+00,2014/01/01 05:00:00+00,2014.0,January,1.0,1.0,Wednesday,Outside,Theft Over,NSA,NSA,-85.488744,0.000000
4,GO-20141261050,2014/01/01 05:00:00+00,2014/01/01 05:00:00+00,2014.0,January,1.0,1.0,Wednesday,Outside,Assault,69,Blake-Jones,-79.344839,43.678946


In [6]:
# Dataframe count and number of NA values
print(f"Number of rows: \n{mci_df.count()}")
print("\n")
print(f"Number of NA values: \n{mci_df.isna().sum()}")

Number of rows: 
EVENT_UNIQUE_ID      323296
REPORT_DATE          323296
OCC_DATE             323296
OCC_YEAR             323191
OCC_MONTH            323191
OCC_DAY              323191
OCC_DOY              323191
OCC_DOW              323191
PREMISES_TYPE        323296
MCI_CATEGORY         323296
HOOD_158             323296
NEIGHBOURHOOD_158    323296
LONG_WGS84           323296
LAT_WGS84            323296
dtype: int64


Number of NA values: 
EVENT_UNIQUE_ID        0
REPORT_DATE            0
OCC_DATE               0
OCC_YEAR             105
OCC_MONTH            105
OCC_DAY              105
OCC_DOY              105
OCC_DOW              105
PREMISES_TYPE          0
MCI_CATEGORY           0
HOOD_158               0
NEIGHBOURHOOD_158      0
LONG_WGS84             0
LAT_WGS84              0
dtype: int64


In [7]:
# Drop null values / rows with no neighbourhood information / Duplicate rows
mci_df = mci_df.dropna()
mci_df = mci_df.loc[mci_df["HOOD_158"] != "NSA"]
mci_df = mci_df.drop_duplicates(subset=["EVENT_UNIQUE_ID"])
mci_df.count()

EVENT_UNIQUE_ID      276993
REPORT_DATE          276993
OCC_DATE             276993
OCC_YEAR             276993
OCC_MONTH            276993
OCC_DAY              276993
OCC_DOY              276993
OCC_DOW              276993
PREMISES_TYPE        276993
MCI_CATEGORY         276993
HOOD_158             276993
NEIGHBOURHOOD_158    276993
LONG_WGS84           276993
LAT_WGS84            276993
dtype: int64

In [8]:
# Get the data types
mci_df.dtypes

EVENT_UNIQUE_ID       object
REPORT_DATE           object
OCC_DATE              object
OCC_YEAR             float64
OCC_MONTH             object
OCC_DAY              float64
OCC_DOY              float64
OCC_DOW               object
PREMISES_TYPE         object
MCI_CATEGORY          object
HOOD_158              object
NEIGHBOURHOOD_158     object
LONG_WGS84           float64
LAT_WGS84            float64
dtype: object

In [9]:
# convert REPORT_DATE and OCC_DATE to date format / OCC_YEAR, OCC_DAY and OCC_DOY to integers
mci_df["REPORT_DATE"]= pd.to_datetime(mci_df["REPORT_DATE"]).dt.date
mci_df["OCC_DATE"]= pd.to_datetime(mci_df["OCC_DATE"]).dt.date
mci_df["OCC_YEAR"] = mci_df["OCC_YEAR"].astype(int)
mci_df["OCC_DAY"] = mci_df["OCC_DAY"].astype(int)
mci_df["OCC_DOY"] = mci_df["OCC_DOY"].astype(int)
mci_df.head()

,EVENT_UNIQUE_ID,REPORT_DATE,OCC_DATE,OCC_YEAR,OCC_MONTH,OCC_DAY,OCC_DOY,OCC_DOW,PREMISES_TYPE,MCI_CATEGORY,HOOD_158,NEIGHBOURHOOD_158,LONG_WGS84,LAT_WGS84
1,GO-20141260701,2014-01-01,2014-01-01,2014,January,1,1,Wednesday,Commercial,Break and Enter,70,South Riverdale,-79.350229,43.646293
2,GO-20141260889,2014-01-01,2014-01-01,2014,January,1,1,Wednesday,Apartment,Assault,74,North St.James Town,-79.376497,43.666423
4,GO-20141261050,2014-01-01,2014-01-01,2014,January,1,1,Wednesday,Outside,Assault,69,Blake-Jones,-79.344839,43.678946
5,GO-20141259344,2014-01-01,2014-01-01,2014,January,1,1,Wednesday,Commercial,Assault,164,Wellington Place,-79.391841,43.646639
6,GO-20141260233,2014-01-01,2014-01-01,2014,January,1,1,Wednesday,Commercial,Break and Enter,126,Dorset Park,-79.281075,43.765796


In [10]:
# Rename columns
mci_df = mci_df.rename(
    columns={"EVENT_UNIQUE_ID": "Event Unique ID",
            "REPORT_DATE": "Report Date",
            "OCC_DATE": "Occurence Date",
            "OCC_YEAR": "Occurence Year",
            "OCC_MONTH": "Occurence Month",
            "OCC_DAY": "Occurence Day",
            "OCC_DOY": "Occurence Day of Year",
            "OCC_DOW": "Occurence Day of Week",
            "PREMISES_TYPE": "Premises Type",
            "MCI_CATEGORY": "MCI Category",
            "HOOD_158": "Hood ID",
            "NEIGHBOURHOOD_158": "Neighbourhood",
            "LONG_WGS84": "Longitude",
            "LAT_WGS84": "Latitude"})

mci_df.head()

,Event Unique ID,Report Date,Occurence Date,Occurence Year,Occurence Month,Occurence Day,Occurence Day of Year,Occurence Day of Week,Premises Type,MCI Category,Hood ID,Neighbourhood,Longitude,Latitude
1,GO-20141260701,2014-01-01,2014-01-01,2014,January,1,1,Wednesday,Commercial,Break and Enter,70,South Riverdale,-79.350229,43.646293
2,GO-20141260889,2014-01-01,2014-01-01,2014,January,1,1,Wednesday,Apartment,Assault,74,North St.James Town,-79.376497,43.666423
4,GO-20141261050,2014-01-01,2014-01-01,2014,January,1,1,Wednesday,Outside,Assault,69,Blake-Jones,-79.344839,43.678946
5,GO-20141259344,2014-01-01,2014-01-01,2014,January,1,1,Wednesday,Commercial,Assault,164,Wellington Place,-79.391841,43.646639
6,GO-20141260233,2014-01-01,2014-01-01,2014,January,1,1,Wednesday,Commercial,Break and Enter,126,Dorset Park,-79.281075,43.765796


In [11]:
# Filter to get data from 2016 to 2022
mci_cleaned_df = mci_df[(mci_df["Occurence Year"]== 2016) | (mci_df["Occurence Year"]== 2017) | (mci_df["Occurence Year"]== 2018) | 
               (mci_df["Occurence Year"]== 2019) | (mci_df["Occurence Year"]== 2020) | (mci_df["Occurence Year"]== 2021) |
                (mci_df["Occurence Year"]== 2022)]
# Reset index
mci_cleaned_df = mci_cleaned_df.reset_index(drop=True)

# output cleaned file to csv
mci_cleaned_df.to_csv("../Output/MCI_main_cleaned_data.csv")

mci_cleaned_df.head()

,Event Unique ID,Report Date,Occurence Date,Occurence Year,Occurence Month,Occurence Day,Occurence Day of Year,Occurence Day of Week,Premises Type,MCI Category,Hood ID,Neighbourhood,Longitude,Latitude
0,GO-20163930,2016-01-01,2016-01-01,2016,January,1,1,Friday,House,Assault,133,Centennial Scarborough,-79.144112,43.776319
1,GO-20164197,2016-01-01,2016-01-01,2016,January,1,1,Friday,Outside,Assault,159,Etobicoke City Centre,-79.546283,43.616413
2,GO-20162690,2016-01-01,2016-01-01,2016,January,1,1,Friday,House,Auto Theft,25,Glenfield-Jane Heights,-79.522923,43.751382
3,GO-2016418,2016-01-01,2016-01-01,2016,January,1,1,Friday,Outside,Assault,70,South Riverdale,-79.354948,43.641216
4,GO-2016452,2016-01-01,2016-01-01,2016,January,1,1,Friday,Apartment,Assault,108,Briar Hill-Belgravia,-79.451926,43.701471


In [12]:
# Group by year, month and date and return the crime count
mci_grouped_year_date = mci_cleaned_df.groupby(["Occurence Year", "Occurence Month", "Occurence Date"]).size()
mci_grouped_year_date_df = mci_grouped_year_date.to_frame()
mci_grouped_year_date_df = mci_grouped_year_date_df.reset_index()
mci_grouped_year_date_df = mci_grouped_year_date_df.sort_values(by = ["Occurence Date"], ascending = True)
mci_grouped_year_date_df = mci_grouped_year_date_df.rename(columns = {0: "Crime Count"})

#output file to csv
mci_grouped_year_date_df.to_csv("../Output/mci_grouped_ymd.csv")

mci_grouped_year_date_df.head()

,Occurence Year,Occurence Month,Occurence Date,Crime Count
121,2016,January,2016-01-01,161
122,2016,January,2016-01-02,66
123,2016,January,2016-01-03,64
124,2016,January,2016-01-04,53
125,2016,January,2016-01-05,72


In [13]:
# Group by year, neighbourhood and crime category and return the crime count
mci_grouped_date_hood = mci_cleaned_df.groupby(["Occurence Year", "Neighbourhood", "MCI Category"]).size()
mci_grouped_date_hood_df = mci_grouped_date_hood.to_frame()
mci_grouped_date_hood_df = mci_grouped_date_hood_df.reset_index()
mci_grouped_date_hood_df = mci_grouped_date_hood_df.rename(columns = {0: "Crime Count"})

#output file to csv
mci_grouped_date_hood_df.to_csv("../Output/mci_grouped_date_hood.csv")

mci_grouped_date_hood_df.head()

,Occurence Year,Neighbourhood,MCI Category,Crime Count
0,2016,Agincourt North,Assault,69
1,2016,Agincourt North,Auto Theft,17
2,2016,Agincourt North,Break and Enter,47
3,2016,Agincourt North,Robbery,12
4,2016,Agincourt North,Theft Over,5
